In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import MissingIndicator # Es una función que permite identificar los datos faltantes
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import NaN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder # Librería donde está el método One-Hot Encoder
from scipy.stats import fisher_exact # Test exacto de Fisher
from sklearn.preprocessing import MinMaxScaler

In [13]:
dir = "C:/Users/alejo/OneDrive/Especializacion/Caso Dyna\Caso-Dyna/B_D/"

df = None

años = ['2021','2022']
meses = ['01','02','03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for i in (años):
  for j in (meses):
    try:
      #dfu = pd.read_csv(f"{dir}{i}{j}.csv", low_memory=False, encoding='latin-1', dayfirst=True, parse_dates=True, index_col=[0]).reset_index()
      dfu = pd.read_csv(f"{dir}{i}{j}.csv", low_memory=False, encoding='latin-1', decimal=".")
      df = pd.concat([df,dfu])
    except:
      pass

In [14]:
df.head()

,Fecha,Nro documento,Estado movto.,Bodega,Item,Desc. item,CLASIFICACION VENTAS PESOS,CLASIFICACION ROTACION UNIDADES,U.M.,Cant. pedida,...,Desc depto,Ciudad,Desc. ciudad,Barrio,Desc. Barrio,Razón social cliente despacho,Referencia,DirecciÃ³n 1,RazÃ³n social cliente despacho,Unnamed: 31
0,01/11/2021,PED-01323412,Cumplido,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,1.0,...,Antioquia,541.0,PEÑOL (281),PRINCIPAL,PRINCIPAL,CATAÑO OSPINA S A S,T1107-0037 ...,NaN,NaN,NaN
1,01/11/2021,PED-01323412,Cumplido,1.0,6958.0,TERMINAL CAFE 1/2pg TCN1/2,0005 - E (100%),0005 - E (100%),B30,1.0,...,Antioquia,541.0,PEÑOL (281),PRINCIPAL,PRINCIPAL,CATAÑO OSPINA S A S,TCN1/2 ...,NaN,NaN,NaN
2,01/04/2021,PED-01321445,Cumplido,1.0,5637.0,CERRADURA FORTE DERECHA BEIGE,0004 - D (80%),0004 - D (80%),UND,24.0,...,Antioquia,490.0,NECOCLI (450),NaN,,ATLANTICO S A S,5637,NaN,NaN,NaN
3,01/19/2021,PED-01326898,Cumplido,1.0,790527.0,CODAL 35X75mm X6m CODALA0167,0002 - B (60%),0005 - E (100%),UND,12.0,...,Antioquia,490.0,NECOCLI (450),NaN,,DEPOSITO AGROPECUARIO DE OCCIDENTE S A S,1009124,NaN,NaN,NaN
4,01/20/2021,PED-01327163,Cumplido,1.0,600643.0,CANAST LAVP ALUM 2pg 580128,0005 - E (100%),0005 - E (100%),UND,6.0,...,Antioquia,490.0,NECOCLI (450),NaN,,DEPOSITO AGROPECUARIO DE OCCIDENTE S A S,101088,NaN,NaN,NaN


In [15]:
df.shape

(2897009, 34)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2897009 entries, 0 to 26991
Data columns (total 34 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Fecha                            object 
 1   Nro documento                    object 
 2   Estado movto.                    object 
 3   Bodega                           float64
 4   Item                             float64
 5   Desc. item                       object 
 6   CLASIFICACION VENTAS PESOS       object 
 7   CLASIFICACION ROTACION UNIDADES  object 
 8   U.M.                             object 
 9   Cant. pedida                     object 
 10  Cant. comprom.                   float64
 11  Cant. factura                    object 
 12  Desc. motivo                     object 
 13  Vendedor                         object 
 14  Nombre vendedor                  object 
 15  Valor bruto                      object 
 16  COMPRAS                          object 
 17  MARCA          

In [17]:
#Se observa que las ultimas columnas de los pedidos se agregaron unas variables desconocidas, se deben revisar.
df = df.rename(columns={'DirecciÃ³n 1':'Direccion_2','RazÃ³n social cliente despacho':'Razon social cliente despacho 2', 'Unnamed: 31': 'variable_31'})

In [18]:
df_2 = df[df['variable_31'].notnull()]
df_2.head()

,Fecha,Nro documento,Estado movto.,Bodega,Item,Desc. item,CLASIFICACION VENTAS PESOS,CLASIFICACION ROTACION UNIDADES,U.M.,Cant. pedida,...,Desc depto,Ciudad,Desc. ciudad,Barrio,Desc. Barrio,Razón social cliente despacho,Referencia,Direccion_2,Razon social cliente despacho 2,variable_31
30000,08/05/2021,PED-01397114,Cumplido,1.0,44617.0,REFLECTOR LED NOVA II 30W LB 25KH ILM CJ,0005 - E (100%),0005 - E (100%),UND,1.0,...,11,Bogotá,1,SANTAFE DE BOGOTA D.C. (120),NaN,,CRUZ LEGUIZAMON PAUL ALEJANDRO,NaN,NaN,1814
36097,08/05/2021,PED-01397114,Cumplido,1.0,44671.0,"ESTANTERIA SOLIDO 15"" 5 NIVELES NEGRA",0005 - E (100%),0005 - E (100%),UND,1.0,...,11,Bogotá,1,SANTAFE DE BOGOTA D.C. (120),NaN,,CRUZ LEGUIZAMON PAUL ALEJANDRO,NaN,NaN,9900
63182,08/05/2021,PED-01397114,Cumplido,1.0,44774.0,TUBO DE DUCHA CUADRADO 40 CM,0005 - E (100%),0005 - E (100%),UND,1.0,...,11,Bogotá,1,SANTAFE DE BOGOTA D.C. (120),NaN,,CRUZ LEGUIZAMON PAUL ALEJANDRO,NaN,NaN,MET-403A ...
72002,08/05/2021,PED-01397114,Cumplido,1.0,44494.0,DUCHA ULTRA DELGADA CUADRADA 20x20,0003 - C (70%),0005 - E (100%),UND,1.0,...,11,Bogotá,1,SANTAFE DE BOGOTA D.C. (120),NaN,,CRUZ LEGUIZAMON PAUL ALEJANDRO,NaN,NaN,MET-335 ...
127773,08/05/2021,PED-01397114,Cumplido,1.0,44371.0,"LIJADORA ROTORBITAL 5"" GEX 125-1 AE",0004 - D (80%),0005 - E (100%),UND,1.0,...,11,Bogotá,1,SANTAFE DE BOGOTA D.C. (120),NaN,,CRUZ LEGUIZAMON PAUL ALEJANDRO,NaN,NaN,0601.387.5G0-000 ...


Es como si la información se corriera a las otras columnas por la cantidad de caracteres en ellas y así se crearon estas variables, porcederemos entonces a eliminar registros y columnas ya que no son un gran porcentaje.

In [19]:
df_2.shape

(13, 34)

In [20]:
#solo se econtraron 13 filas con información en esa variable, se procede a eliminar los registros y las columnas

#Borrar registros
df.drop(df[df['variable_31'].notnull()].index, inplace = True)

#Borrar columnas
df.drop(['variable_31'], axis=1, inplace=True)

In [21]:
#Revisisón de las otras dos variables.
df_2 = df[df['Direccion_2'].notnull() | df['Razon social cliente despacho 2'].notnull()]
df_2.head()

,Fecha,Nro documento,Estado movto.,Bodega,Item,Desc. item,CLASIFICACION VENTAS PESOS,CLASIFICACION ROTACION UNIDADES,U.M.,Cant. pedida,...,Depto,Desc depto,Ciudad,Desc. ciudad,Barrio,Desc. Barrio,Razón social cliente despacho,Referencia,Direccion_2,Razon social cliente despacho 2
0,2/26/2021,PED-01341848,Cumplido,1.0,733543.0,SEGUETA BIMETAL 18DPP 67311800 NICHOLSON,0001 - A (50%),0001 - A (50%),E150,2,...,20.0,Cesar,1.0,VALLEDUPAR (448),NaN,,NaN,NF1218B ...,TRANSV 26 16 B 08 VILLA CORELCA,SOLANO MOLINA DANIELA SOFIA
1,2/5/2021,PED-01333770,Cumplido,1.0,750877.0,"BROCA PARA CONCRETO 1/4"" X 4"" INCOLMA",0001 - A (50%),0001 - A (50%),UND,36,...,20.0,Cesar,1.0,VALLEDUPAR (448),NaN,,NaN,89410608,MZ 15 CA 13 BR VILLA MYRIAM,INVERSIONES JK & MK S A S
2,2/2/2021,PED-01332381,Aprobado,1.0,940346.0,ALICATE DIABLO 10pg 84-221LA STANLEY,0001 - A (50%),0004 - D (80%),UND,3,...,23.0,CÃ³rdoba,1.0,MONTERIA (250),CENTRO,CENTRO,NaN,84-221LA ...,CR 2 38 06,HERNANDEZ JARAMILLO ELDA ROSA
3,2/2/2021,PED-01332381,Cumplido,1.0,43346.0,DISCO FINO T41 4.1/2X0.045X7/8 DW8062:C,0001 - A (50%),0001 - A (50%),E100,1,...,23.0,CÃ³rdoba,1.0,MONTERIA (250),CENTRO,CENTRO,NaN,DW8062:C ...,CR 2 38 06,HERNANDEZ JARAMILLO ELDA ROSA
4,2/10/2021,PED-01335384,Cumplido,1.0,7413.0,ASIENTO SANIT BASIC BONE X3 869713331,0001 - A (50%),0005 - E (100%),E12,1,...,23.0,CÃ³rdoba,1.0,MONTERIA (250),CENTRO,CENTRO,NaN,869713331,CL 36 4 64,LOBO ARGUMEDO ARGEMIRO ANTONIO


Al parece al concatenar la información la dirección y la razon social no coincidieron para unirse como se debía.

In [22]:
#Generar nuevas variables para estas dos situaciones
condition = [
    (df['Razón social cliente despacho'].isnull())]
choice = [df['Razon social cliente despacho 2']]
df['Razon_social'] = np.select(condition, choice, default=df['Razón social cliente despacho'])

In [23]:
condition2 = [
    (df['Dirección 1'].isnull())]
choice2 = [df['Direccion_2']]
df['Direccion'] = np.select(condition2, choice2, default=df['Dirección 1'])

In [24]:
#Porcedemos a eliminar las vbles con nulos y nos quedamos con las que creamos

#Borrar columna
df.drop(['Dirección 1'], axis=1, inplace=True)
df.drop(['Direccion_2'], axis=1, inplace=True)
df.drop(['Razón social cliente despacho'], axis=1, inplace=True)
df.drop(['Razon social cliente despacho 2'], axis=1, inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2896733 entries, 0 to 26991
Data columns (total 31 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Fecha                            object 
 1   Nro documento                    object 
 2   Estado movto.                    object 
 3   Bodega                           float64
 4   Item                             float64
 5   Desc. item                       object 
 6   CLASIFICACION VENTAS PESOS       object 
 7   CLASIFICACION ROTACION UNIDADES  object 
 8   U.M.                             object 
 9   Cant. pedida                     object 
 10  Cant. comprom.                   float64
 11  Cant. factura                    object 
 12  Desc. motivo                     object 
 13  Vendedor                         object 
 14  Nombre vendedor                  object 
 15  Valor bruto                      object 
 16  COMPRAS                          object 
 17  MARCA          

In [26]:
#Se debe revisar el tipo de variable, sobretodo las númerica que se están cargando como de tipo objeto
df["Cant. pedida"] = [float(str(i).replace(",", "")) for i in df["Cant. pedida"]]
df["Cant. factura"] = [float(str(i).replace(",", "")) for i in df["Cant. factura"]]
df['Valor bruto'] = [float(str(i).replace(",", "").replace("$", "")) for i in df['Valor bruto']]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2896733 entries, 0 to 26991
Data columns (total 31 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Fecha                            object 
 1   Nro documento                    object 
 2   Estado movto.                    object 
 3   Bodega                           float64
 4   Item                             float64
 5   Desc. item                       object 
 6   CLASIFICACION VENTAS PESOS       object 
 7   CLASIFICACION ROTACION UNIDADES  object 
 8   U.M.                             object 
 9   Cant. pedida                     float64
 10  Cant. comprom.                   float64
 11  Cant. factura                    float64
 12  Desc. motivo                     object 
 13  Vendedor                         object 
 14  Nombre vendedor                  object 
 15  Valor bruto                      float64
 16  COMPRAS                          object 
 17  MARCA          

Ahora que tenemos una base de datos más limpia, vamos a filtrar por los ítems que son importados que son los cuales presentan una mayor problematica para la empresa, sin embargo esta información no está en los datos de pedidos por lo tanto debemos unirlo con el maestro de ítems.

In [29]:
maestro = pd.read_csv("C:/Users/alejo/OneDrive/Especializacion/Caso Dyna/Caso-Dyna/B_D/Maestro.csv", encoding='latin-1')

In [30]:
df = df.merge(maestro, on="Item", how='inner')
df.head()

,Fecha,Nro documento,Estado movto.,Bodega,Item,Desc. item,CLASIFICACION VENTAS PESOS,CLASIFICACION ROTACION UNIDADES,U.M.,Cant. pedida,...,Desc. ciudad,Barrio,Desc. Barrio,Referencia,Razon_social,Direccion,ORIGEN PROVEEDOR,PROVEEDOR CCIAL_y,Estado,COMPRAS_y
0,01/11/2021,PED-01323412,Cumplido,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,1.0,...,PEÑOL (281),PRINCIPAL,PRINCIPAL,T1107-0037 ...,CATAÑO OSPINA S A S,CL 10 CR 11 64,0001 - NACIONAL,0087 - INDUMA,Activo,0001 - SI
1,01/27/2021,PED-01330164,Cumplido,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,2.0,...,DON MATIAS (250),NaN,,T1107-0037 ...,GOMEZ VASQUEZ LINA MARCELA,CL 28 28 42,0001 - NACIONAL,0087 - INDUMA,Activo,0001 - SI
2,01/27/2021,PED-01330248,Aprobado,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,3.0,...,REMEDIOS (287) S.L,LA SALIDA,LA SALIDA,T1107-0037 ...,RESTREPO ECHAVARRIA NELSON,CR 10 6 69,0001 - NACIONAL,0087 - INDUMA,Activo,0001 - SI
3,01/18/2021,PED-01326117,Cumplido,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,5.0,...,POPAYAN (378) S.L,CADILLAL,CADILLAL,T1107-0037 ...,PEREZ QUIJANO MARIA CECILIA,CR 11 2 08 ESQUINA,0001 - NACIONAL,0087 - INDUMA,Activo,0001 - SI
4,01/05/2021,PED-01321993,Cumplido,1.0,8354.0,BISAG OMEGA LAT CT C14 3pg IND,0001 - A (50%),0004 - D (80%),C24,1.0,...,TARSO (250),NaN,,T1107-0037 ...,BEDOYA RUIZ MARLENY DEL SOCORRO,CL 20 12 14,0001 - NACIONAL,0087 - INDUMA,Activo,0001 - SI


In [31]:
#Se agregaron dos columnas que ya estaban dentro del dataset entonces se procede a borrarlas
df.drop(['PROVEEDOR CCIAL_y'], axis=1, inplace=True)
df.drop(['COMPRAS_y'], axis=1, inplace=True)

Nos quedaremos entonces con los pedidos de ítem que son importados por la empresa y que además estén activos porque no tienen sentido hacer un estudio sobre ítems que ya no se encuntran en el portafolio.

In [32]:
df['ORIGEN PROVEEDOR'].unique().tolist()

['0001 - NACIONAL', '0002 - IMPORTADO', '9999 - SIN IDENTIFICAR']

In [33]:
#Borrar registros
df.drop(df[df['Estado'] == 'Bloqueado'].index, inplace = True)
df.drop(df[df['ORIGEN PROVEEDOR'] != '0002 - IMPORTADO'].index, inplace = True)

Dentro de la regla de negocio, no se deben tener en cuenta los pedidos anulados, en elaboración, retenidos o nan ya que estos pedidos no llegaron a hacer nunca una demanda formal y no se gestionaron, se deben eliminar esos registros.

In [34]:
df['Estado movto.'].unique().tolist()

['Cumplido',
 'Aprobado',
 'Anulado',
 'En elaboraciÃ³n',
 'En elaboración',
 'Retenido',
 'Comprometido']

In [35]:
#Borrar registros
df.drop(df[(df['Estado movto.'] == 'Anulado') | (df['Estado movto.'] == 'En elaboración') | (df['Estado movto.'] == 'En elaboraciÃ³n') | (df['Estado movto.'] == 'Retenido')].index, inplace=True)

In [36]:
df.shape

(923249, 33)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 923249 entries, 21687 to 2896653
Data columns (total 33 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Fecha                            923249 non-null  object 
 1   Nro documento                    923249 non-null  object 
 2   Estado movto.                    923249 non-null  object 
 3   Bodega                           923249 non-null  float64
 4   Item                             923249 non-null  float64
 5   Desc. item                       923249 non-null  object 
 6   CLASIFICACION VENTAS PESOS       923249 non-null  object 
 7   CLASIFICACION ROTACION UNIDADES  923249 non-null  object 
 8   U.M.                             923249 non-null  object 
 9   Cant. pedida                     923249 non-null  float64
 10  Cant. comprom.                   923249 non-null  float64
 11  Cant. factura                    923249 non-null  float64
 12  De

Se observa presencia de datos nulos

In [38]:
# Reemplazamos los datos faltantes por datos NaN o nulos
df = df.replace(" ",np.nan)
df = df.replace("",np.nan) 
df = df.replace("?",np.nan) 

In [39]:
Indicador = MissingIndicator(missing_values = np.nan) # Decimos que datos debe buscar como datos faltantes
Indicador.fit(df) # Aplicamos el método de búsqueda con nuestra base de datos
Datos_Indicador = Indicador.transform(df) # Realizamos una transformación Booleana de los datos para identificar los datos faltantes (True)
                                             # y los datos no faltantes (False)
display(Datos_Indicador) # Mostramos los resultados de la búsqueda de los datos faltantes y no faltantes

array([[ True,  True, False],
       [ True,  True, False],
       [ True,  True, False],
       ...,
       [ True,  True, False],
       [ True,  True, False],
       [ True,  True, False]])

In [40]:
print("Características donde hay datos faltantes: ", Indicador.features_)

Características donde hay datos faltantes:  [26 27 30]


In [41]:
POS = np.where(Datos_Indicador == True) # Se busca aquellos datos que tiene un valor Booleanos igual a True
print("Porcentaje de Datos Faltantes (%): ", 100*(len(POS[0])/(Datos_Indicador.shape[0]*Datos_Indicador.shape[1])))

Porcentaje de Datos Faltantes (%):  37.914943133795255


Es un porcentaje de datos nulos muy alto, no podemos simplemente eliminar todos estos registros, vamos a revisar puntualmente

In [42]:
#Revsisón de variables puntuales direccion
df1 = df[df['Direccion'].isnull()]
df1.shape

(8, 33)

In [43]:
#Borrar registros
df.drop(df[df['Direccion'].isnull()].index, inplace=True)

In [44]:
#Revsisón de variables puntuales Barrio y Desc Barrio
df1 = df[df['Barrio'].isnull() | df['Desc. Barrio'].isnull()]
df1.shape

(525378, 33)

Hay demasiados registros con datos nulos dentro de la base de datos, y para esto se tiene la opción de eliminar o imputar, sin embargo al ser tantos datos perderías mucha información si decidieramos eliminar; y por otro lado si se imputa habrían registros sin sentido como por ejemplo barrios que no existen dentro de un departamanto. Se optara entonces por reemplazarlos por el valor "Desconocido" teniendo en cuenta que estas son variable categóricas

In [45]:
#Reemplazar valores nulos por "Desconocido"
df['Barrio'].fillna("Desconocido", inplace = True)
df['Desc. Barrio'].fillna("Desconocido", inplace = True)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 923241 entries, 21687 to 2896653
Data columns (total 33 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Fecha                            923241 non-null  object 
 1   Nro documento                    923241 non-null  object 
 2   Estado movto.                    923241 non-null  object 
 3   Bodega                           923241 non-null  float64
 4   Item                             923241 non-null  float64
 5   Desc. item                       923241 non-null  object 
 6   CLASIFICACION VENTAS PESOS       923241 non-null  object 
 7   CLASIFICACION ROTACION UNIDADES  923241 non-null  object 
 8   U.M.                             923241 non-null  object 
 9   Cant. pedida                     923241 non-null  float64
 10  Cant. comprom.                   923241 non-null  float64
 11  Cant. factura                    923241 non-null  float64
 12  De

In [47]:
#Modificar el tipo de variables

df['Estado movto.']=df['Estado movto.'].astype('category')
df['Nro documento']=df['Nro documento'].astype('string')
df['Bodega']=df['Bodega'].astype('string')
df['Item']=df['Item'].astype('string')
df['Desc. item']=df['Desc. item'].astype('string')
df['CLASIFICACION VENTAS PESOS']=df['CLASIFICACION VENTAS PESOS'].astype('category')
df['CLASIFICACION ROTACION UNIDADES']=df['CLASIFICACION ROTACION UNIDADES'].astype('category')
df['Desc. motivo']=df['Desc. motivo'].astype('category')
df['U.M.']=df['U.M.'].astype('string')
df['Vendedor']=df['Vendedor'].astype('string')
df['Nombre vendedor']=df['Nombre vendedor'].astype('string')
df['COMPRAS_x']=df['COMPRAS_x'].astype('category')
df['PROVEEDOR CCIAL_x']=df['PROVEEDOR CCIAL_x'].astype('category')
df['FAMILIA-PRODUCTO']=df['FAMILIA-PRODUCTO'].astype('category')
df['MARCA']=df['MARCA'].astype('category')
df['CATEGORIA']=df['CATEGORIA'].astype('category')
df['SUBCATEGORIA']=df['SUBCATEGORIA'].astype('category')
df['Depto']=df['Depto'].astype('category')
df['Desc depto']=df['Desc depto'].astype('category')
df['Ciudad']=df['Ciudad'].astype('category')
df['Desc. ciudad']=df['Desc. ciudad'].astype('category')
df['Barrio']=df['Barrio'].astype('category')
df['Desc. Barrio']=df['Desc. Barrio'].astype('category')
df['Razon_social']=df['Razon_social'].astype('string')
df['Referencia']=df['Referencia'].astype('string')
df['ORIGEN PROVEEDOR']=df['ORIGEN PROVEEDOR'].astype('category')
df['Direccion']=df['Direccion'].astype('string')
df['Estado']=df['Estado'].astype('category')

In [48]:
df.dtypes

Fecha                                      object
Nro documento                      string[python]
Estado movto.                            category
Bodega                             string[python]
Item                               string[python]
Desc. item                         string[python]
CLASIFICACION VENTAS PESOS               category
CLASIFICACION ROTACION UNIDADES          category
U.M.                               string[python]
Cant. pedida                              float64
Cant. comprom.                            float64
Cant. factura                             float64
Desc. motivo                             category
Vendedor                           string[python]
Nombre vendedor                    string[python]
Valor bruto                               float64
COMPRAS_x                                category
MARCA                                    category
CATEGORIA                                category
SUBCATEGORIA                             category
